In [31]:
from common_imports import*

In [32]:
data = '/Applications/Kaushik/North Texas/Education/Semester 5, Fall 2024/Project/Fresh start/CSV/q2_with_benefit.csv'
df = pd.read_csv(data)

In [33]:
# Subtract 'Column2' from 'Column1' and store the result in a new column 'Result'
df['diff_delivery'] = df['Days for shipping (real)'] - df['Days for shipment (scheduled)']

In [34]:
print(df.head())

   Days for shipping (real)  Days for shipment (scheduled)  Benefit per order  \
0                         3                              4          91.250000   
1                         5                              4        -249.089996   
2                         4                              4        -247.779999   
3                         3                              4          22.860001   
4                         2                              4         134.210007   

  Department Name        Market  diff_delivery  
0         Fitness  Pacific Asia             -1  
1         Fitness  Pacific Asia              1  
2         Fitness  Pacific Asia              0  
3         Fitness  Pacific Asia             -1  
4         Fitness  Pacific Asia             -2  


In [35]:
# Create a new column with if-else logic
df['late_delivery'] = np.where(df['diff_delivery'] < 0, 'On Time', 'Late')

print(df.head())

   Days for shipping (real)  Days for shipment (scheduled)  Benefit per order  \
0                         3                              4          91.250000   
1                         5                              4        -249.089996   
2                         4                              4        -247.779999   
3                         3                              4          22.860001   
4                         2                              4         134.210007   

  Department Name        Market  diff_delivery late_delivery  
0         Fitness  Pacific Asia             -1       On Time  
1         Fitness  Pacific Asia              1          Late  
2         Fitness  Pacific Asia              0          Late  
3         Fitness  Pacific Asia             -1       On Time  
4         Fitness  Pacific Asia             -2       On Time  


In [36]:
df_copy = df.drop(['Days for shipping (real)','Days for shipment (scheduled)'],axis = 1)

print(df_copy.head())

   Benefit per order Department Name        Market  diff_delivery  \
0          91.250000         Fitness  Pacific Asia             -1   
1        -249.089996         Fitness  Pacific Asia              1   
2        -247.779999         Fitness  Pacific Asia              0   
3          22.860001         Fitness  Pacific Asia             -1   
4         134.210007         Fitness  Pacific Asia             -2   

  late_delivery  
0       On Time  
1          Late  
2          Late  
3       On Time  
4       On Time  


In [37]:
df_copy = df_copy.drop('diff_delivery', axis = 1)

In [38]:
print(df_copy.head())

   Benefit per order Department Name        Market late_delivery
0          91.250000         Fitness  Pacific Asia       On Time
1        -249.089996         Fitness  Pacific Asia          Late
2        -247.779999         Fitness  Pacific Asia          Late
3          22.860001         Fitness  Pacific Asia       On Time
4         134.210007         Fitness  Pacific Asia       On Time


In [40]:
# Assuming your dataset is loaded into a variable named 'data'

# Group by 'Category Name' and 'Customer City' to calculate late delivery stats
late_delivery_analysis = df_copy.groupby(['Department Name', 'Market']).agg(
    total_orders=('late_delivery', 'count'),
    late_orders=('late_delivery', lambda x: (x == 'Late').sum()),
    on_time_orders=('late_delivery', lambda x: (x == 'On Time').sum())
).reset_index()

# Calculate the proportion of late deliveries
late_delivery_analysis['late_percentage'] = (
    late_delivery_analysis['late_orders'] / late_delivery_analysis['total_orders'] * 100
)

# Sort by the highest late delivery percentage
late_delivery_analysis = late_delivery_analysis.sort_values(by='late_percentage', ascending=False)

# Display the results
print(late_delivery_analysis)

       Department Name        Market  total_orders  late_orders  \
35            Pet Shop  Pacific Asia           492          397   
30            Outdoors        Africa           632          496   
17             Fitness  Pacific Asia           780          608   
5            Book Shop        Europe           308          238   
28                Golf          USCA          5082         3917   
0              Apparel        Africa          3162         2434   
7           Discs Shop        Europe           348          267   
29  Health and Beauty   Pacific Asia           362          277   
4              Apparel          USCA          7111         5440   
22            Footwear  Pacific Asia          3021         2308   
3              Apparel  Pacific Asia         11103         8480   
10            Fan Shop        Europe         18265        13945   
24                Golf        Africa          2219         1694   
19            Footwear        Africa           985          75

In [41]:
print(late_delivery_analysis.head())

   Department Name        Market  total_orders  late_orders  on_time_orders  \
35        Pet Shop  Pacific Asia           492          397              95   
30        Outdoors        Africa           632          496             136   
17         Fitness  Pacific Asia           780          608             172   
5        Book Shop        Europe           308          238              70   
28            Golf          USCA          5082         3917            1165   

    late_percentage  
35        80.691057  
30        78.481013  
17        77.948718  
5         77.272727  
28        77.075954  


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Encode categorical variables
df_encoded = pd.get_dummies(df_copy, columns=['Department Name', 'Market'], drop_first=True)

# Define features and target
X = df_encoded.drop(['late_delivery'], axis=1)
y = df_copy['late_delivery'].apply(lambda x: 1 if x == 'Late' else 0)  # Binary target encoding

# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Add predictions to the dataset
df_copy['predicted_delivery'] = model.predict(df_encoded.drop(['late_delivery'], axis=1))
df_copy['predicted_delivery'] = df_copy['predicted_delivery'].apply(lambda x: 'Late' if x == 1 else 'On Time')

# Save the updated dataset
df_copy.to_csv('shipment_with_predictions.csv', index=False)

Accuracy: 0.6746620873033459

Classification Report:
               precision    recall  f1-score   support

           0       0.24      0.17      0.20     12921
           1       0.76      0.83      0.80     41235

    accuracy                           0.67     54156
   macro avg       0.50      0.50      0.50     54156
weighted avg       0.64      0.67      0.65     54156



In [52]:
from sklearn.linear_model import LogisticRegression

# Train the Logistic Regression model
model = LogisticRegression()
model.fit(X, y)

# Get the coefficients (weights) and intercept
coefficients = model.coef_[0]
intercept = model.intercept_[0]

# Create the regression equation
equation = "logit(p) = " + str(intercept)
for feature, coef in zip(X.columns, coefficients):
    equation += f" + ({coef:.4f} * {feature})"

print(f"Regression Equation: {equation}")


Regression Equation: logit(p) = 1.1380698378191085 + (-0.0000 * Benefit per order) + (0.0124 * Department Name_Book Shop) + (-0.0059 * Department Name_Discs Shop) + (0.0001 * Department Name_Fan Shop) + (-0.0011 * Department Name_Fitness) + (-0.0205 * Department Name_Footwear) + (-0.0178 * Department Name_Golf) + (0.0079 * Department Name_Health and Beauty ) + (-0.0730 * Department Name_Outdoors) + (0.0644 * Department Name_Pet Shop) + (-0.0016 * Department Name_Technology) + (0.0218 * Market_Europe) + (0.0141 * Market_LATAM) + (0.0257 * Market_Pacific Asia) + (0.0599 * Market_USCA)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [51]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Grid search
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='f1_weighted')
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


In [53]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Oversample using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Train Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6517279930005346

Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.62      0.64     41229
           1       0.64      0.68      0.66     41063

    accuracy                           0.65     82292
   macro avg       0.65      0.65      0.65     82292
weighted avg       0.65      0.65      0.65     82292



In [47]:
print(df_copy.head())

   Benefit per order Department Name        Market late_delivery  \
0          91.250000         Fitness  Pacific Asia       On Time   
1        -249.089996         Fitness  Pacific Asia          Late   
2        -247.779999         Fitness  Pacific Asia          Late   
3          22.860001         Fitness  Pacific Asia       On Time   
4         134.210007         Fitness  Pacific Asia       On Time   

  predicted_delivery  
0               Late  
1               Late  
2               Late  
3            On Time  
4               Late  
